In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [3]:
train_dir = "../input/10-monkey-species/training/training"
valid_dir = "../input/10-monkey-species/validation/validation"
label_file = "../input/10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))


In [4]:
labels=pd.read_csv(label_file,header=0)
print(labels)

In [37]:
height=128
width=128
channels=3
batch_size=16
num_classes=10


In [6]:
#相当于是一个图像预处理器
train_datagen=keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [7]:
train_generator=train_datagen.flow_from_directory(train_dir,
                                                 target_size=(height,width),
                                                 batch_size=batch_size,
                                                 seed=7,
                                                 shuffle=True,
                                                 class_mode='categorical')


In [8]:
valid_datagen=keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator=valid_datagen.flow_from_directory(valid_dir,
                                                 target_size=(height,width),
                                                 seed=7,
                                                 shuffle=False,
                                                 class_mode='categorical')


In [9]:
train_num=train_generator.samples
valid_num=valid_generator.samples
print(train_num,valid_num)

In [10]:
for i in range(2):
    x,y=train_generator.next()
    print(x.shape,y.shape)
    print(y)

In [11]:
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                       activation='relu',input_shape=[width,height,channels]),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(num_classes,activation='softmax')
    
]
)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy'])
model.summary()

In [19]:
len(model.layers)

In [29]:
epochs = 10
history=model.fit_generator(train_generator,
                            steps_per_epoch=train_num//batch_size,
                            epochs=epochs,
                            validation_data=valid_generator,
                            validation_steps=valid_num//batch_size
                           )

In [12]:
def plot_learning_curves(history, label, epcohs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
#因为accuracy的值和loss值的范围很不一样，因此我们打印两条曲线






In [ ]:
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 1.5, 2.5)

In [30]:
model_sep = keras.models.Sequential([
    #第一组卷积
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu', input_shape=[width, height, channels]),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    #第二组翻倍
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    #第三组再翻倍
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    #展平后，和全连接层做连接
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model_sep.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [31]:
epochs=10
history_sep = model_sep.fit_generator(train_generator,
                            steps_per_epoch=train_num//batch_size,
                            epochs=epochs,
                            validation_data=valid_generator,
                            validation_steps=valid_num//batch_size,
                           )

In [ ]:
plot_learning_curves(history_sep, 'accuracy', epochs, 0, 1)
plot_learning_curves(history_sep, 'loss', epochs, 1.5, 2.5)